## Building a tool to retrieve all libraries used by all code env on the whole instance

#### 1. Libraries importation

In [17]:
import dataiku
from dataiku import pandasutils as pdu
import pandas as pd

#### 2. Creating a client from inside DSS
To work with the API, a connection needs to be established with DSS, by creating a DSSClient object. Once the connection is established, the DSSClient object serves as the entry point to the other calls.

The Python client can be used from inside DSS. In that case:

- It’s preinstalled, you don’t need to do anything

- You don’t need to provide any API key, as the API client will automatically inherit connection credentials from the current context

In [18]:
client = dataiku.api_client()
# client is now a DSSClient and can perform all authorized actions.

#### 3. Listing all code envs setup on the DSS instance with their libraries

In [62]:
# Get the list of all the code env on the DSS instance
# Every code env is described by a dictionary from which we want to extract code env names
list_code_env_full = client.list_code_envs()

In [64]:
# Filter the code env names and types
list_code_env = []
for code_env in list_code_env_full: 
    temp_code_env = {}
    for key, value in code_env.items():
        if (key == 'envName') or (key == 'envLang'):
            temp_code_env[key] = value
    list_code_env.append(temp_code_env)

In [80]:
# Get the libraries of each code env
for code_env in list_code_env:
    code_env_handle = client.get_code_env(code_env['envLang'], code_env['envName'])
    lib_list = list(filter(None, code_env_handle.get_definition()['actualPackageList'].split('\n')))
    mandatory_lib_list = list(filter(None, code_env_handle.get_definition()['mandatoryPackageList'].split('\n')))
    
    #Remove mandatory libraries from the list
    mandatory_lib_name = [lib.split('==')[0] for lib in mandatory_lib_list]
    final_lib_list = [lib for lib in lib_list if lib.split("==")[0] not in mandatory_lib_name]

In [83]:
code_env

{'envName': 'R-363', 'envLang': 'R'}

In [70]:
sel = client.get_code_env('PYTHON', 'code_env_cc_fraud')

In [77]:
sel =list(filter(None, sel.get_definition()['actualPackageList'].split('\n')))
[lib.split('==')[0] for lib in sel]

['appnope',
 'backcall',
 'certifi',
 'charset-normalizer',
 'decorator',
 'idna',
 'ipykernel',
 'ipython',
 'ipython-genutils',
 'jedi',
 'jupyter-client',
 'jupyter-core',
 'numpy',
 'pandas',
 'parso',
 'pexpect',
 'pickleshare',
 'prompt-toolkit',
 'ptyprocess',
 'Pygments',
 'python-dateutil',
 'pytz',
 'pyzmq',
 'requests',
 'simplegeneric',
 'six',
 'tornado',
 'traitlets',
 'urllib3',
 'wcwidth']

In [19]:
project = client.get_project("DSPIPELINEFRAUD")

In [26]:
project_code_env = project.get_settings().get_raw()['settings']['codeEnvs']['python']['envName']

In [54]:
project_code_env

'code_env_cc_fraud'

In [55]:
project_recipes = project.list_recipes()
project_python_recipe = [recipe for recipe in project_recipes if recipe['type'] == 'python'] 

In [56]:
python_recipe_code_env = [python_recipe['params']['envSelection']['envName'] for python_recipe in project_python_recipe if python_recipe['params']['envSelection']['envMode'] == 'EXPLICIT_ENV']

In [57]:
code_env = client.get_code_env('PYTHON', project_code_env)

In [58]:
code_env.get_definition()['actualPackageList'].split('\n')

['appnope==0.1.3',
 'backcall==0.2.0',
 'certifi==2022.5.18.1',
 'charset-normalizer==2.0.12',
 'decorator==4.4.2',
 'idna==3.3',
 'ipykernel==4.8.2',
 'ipython==7.16.3',
 'ipython-genutils==0.2.0',
 'jedi==0.17.2',
 'jupyter-client==5.2.4',
 'jupyter-core==4.4.0',
 'numpy==1.19.5',
 'pandas==1.0.5',
 'parso==0.7.1',
 'pexpect==4.8.0',
 'pickleshare==0.7.5',
 'prompt-toolkit==3.0.29',
 'ptyprocess==0.7.0',
 'Pygments==2.12.0',
 'python-dateutil==2.8.1',
 'pytz==2020.5',
 'pyzmq==18.0.2',
 'requests==2.27.1',
 'simplegeneric==0.8.1',
 'six==1.16.0',
 'tornado==5.1.1',
 'traitlets==4.3.3',
 'urllib3==1.26.9',
 'wcwidth==0.2.5',
 '']

In [0]:
# list_jupyter_notebooks
# list_recipes
# list_running_notebooks